In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
#This script plots WNP TC tracks in the 7 days before specified dates (e.g. AR-dominant EP days)

In [ ]:
ds=xr.open_dataset(dir+'ls_extreme_rain_taiwan_99_80.nc')
p=ds['large_scale_extreme_rain_all']#total rainfall on ER days
dates=pd.DatetimeIndex(ds.time.values)
dates=pd.DatetimeIndex(dates).values
print(dates)

ds6=xr.open_dataset(dir+'IBTrACS.WP.v04r00.nc')
print(ds6.lat.description)
wind_tc=ds6.usa_wind
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_wp_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not
print(nature_tc)
print(hours_tc)

In [ ]:
ds4=xr.open_dataset(dir+'taiwan_ep_days_stats_var95_newar.nc')
ar_yesno4=ds4['ar_yesno'].values.tolist()
ivt_yesno4=ds4['ivt_yesno'].values.tolist()
tc_yesno4=ds4['tc_yesno'].values.tolist()
other_yesno4=ds4['other_yesno'].values.tolist()
tc_linked_ar_yesno4=ds4['tc_linked_ar_yesno'].values.tolist()
tc_linked_ivt_yesno4=ds4['tc_linked_ivt_yesno'].values.tolist()
tc_remnant_linked_ivt_yesno4=ds4['ivt_tc_remnants_yesno'].values.tolist()
tc_remnant_ar_combo_yesno4=ds4['ar_tc_remnant_combo_yesno'].values.tolist()
tc_remnant_linked_ar_yesno4=ds4['tc_remnant_linked_ar_yesno'].values.tolist()
tc_remnants_yesno4=ds4['tc_remnants_yesno'].values.tolist()
tc_ar_combo_yesno4=ds4['tc_ar_combo_yesno'].values.tolist()

zipped_t1=list(zip(dates,ar_yesno4))#ar-related = green
zipped_t2=list(zip(dates,tc_linked_ar_yesno4))
zipped_t3=list(zip(dates,tc_remnant_linked_ar_yesno4))
zipped_t4=list(zip(dates,tc_yesno4))#tc-related = blue
zipped_t5=list(zip(dates,tc_ar_combo_yesno4))
zipped_t6=list(zip(dates,tc_remnant_ar_combo_yesno4))
zipped_t7=list(zip(dates,tc_remnants_yesno4))
zipped_t8=list(zip(dates,ivt_yesno4))#other IVT-related = grey
zipped_t9=list(zip(dates,tc_linked_ivt_yesno4))
zipped_t10=list(zip(dates,tc_remnant_linked_ivt_yesno4))
zipped_t11=list(zip(dates,other_yesno4))#unspecified = brown

dates_t1=[x[0] for x in zipped_t1 if x[1]==1]
dates_t2=[x[0] for x in zipped_t2 if x[1]==1]
dates_t3=[x[0] for x in zipped_t3 if x[1]==1]
dates_t4=[x[0] for x in zipped_t4 if x[1]==1]
dates_t5=[x[0] for x in zipped_t5 if x[1]==1]
dates_t6=[x[0] for x in zipped_t6 if x[1]==1]
dates_t7=[x[0] for x in zipped_t7 if x[1]==1]
dates_t8=[x[0] for x in zipped_t8 if x[1]==1]
dates_t9=[x[0] for x in zipped_t9 if x[1]==1]
dates_t10=[x[0] for x in zipped_t10 if x[1]==1]
dates_t11=[x[0] for x in zipped_t11 if x[1]==1]

dates_ar=dates_t1+dates_t2+dates_t3
dates_tc=dates_t4+dates_t5+dates_t6+dates_t7
dates_other=dates_t8+dates_t9+dates_t10+dates_t11

dates_t1=pd.DatetimeIndex(dates_t1)
dates_t2=pd.DatetimeIndex(dates_t2)
dates_t3=pd.DatetimeIndex(dates_t3)
dates_t4=pd.DatetimeIndex(dates_t4)
dates_t5=pd.DatetimeIndex(dates_t5)
dates_t6=pd.DatetimeIndex(dates_t6)
dates_t7=pd.DatetimeIndex(dates_t7)
dates_t8=pd.DatetimeIndex(dates_t8)
dates_t9=pd.DatetimeIndex(dates_t9)
dates_t10=pd.DatetimeIndex(dates_t10)
dates_t11=pd.DatetimeIndex(dates_t11)

dates_ar=pd.DatetimeIndex(dates_ar)
dates_tc=pd.DatetimeIndex(dates_tc)
dates_other=pd.DatetimeIndex(dates_other)

dates_t1=[x+dt.timedelta(hours=-12) for x in dates_t1]
dates_t2=[x+dt.timedelta(hours=-12) for x in dates_t2]
dates_t3=[x+dt.timedelta(hours=-12) for x in dates_t3]
dates_t4=[x+dt.timedelta(hours=-12) for x in dates_t4]
dates_t5=[x+dt.timedelta(hours=-12) for x in dates_t5]
dates_t6=[x+dt.timedelta(hours=-12) for x in dates_t6]
dates_t7=[x+dt.timedelta(hours=-12) for x in dates_t7]
dates_t8=[x+dt.timedelta(hours=-12) for x in dates_t8]
dates_t9=[x+dt.timedelta(hours=-12) for x in dates_t9]
dates_t10=[x+dt.timedelta(hours=-12) for x in dates_t10]
dates_t11=[x+dt.timedelta(hours=-12) for x in dates_t11]

dates_tc=[x+dt.timedelta(hours=-12) for x in dates_tc]
dates_ar=[x+dt.timedelta(hours=-12) for x in dates_ar]
dates_other=[x+dt.timedelta(hours=-12) for x in dates_other]

years_t1=[x.year for x in dates_t1]
years_t2=[x.year for x in dates_t2]
years_t3=[x.year for x in dates_t3]
years_t4=[x.year for x in dates_t4]
years_t5=[x.year for x in dates_t5]
years_t6=[x.year for x in dates_t6]
years_t7=[x.year for x in dates_t7]
years_t8=[x.year for x in dates_t8]
years_t9=[x.year for x in dates_t9]
years_t10=[x.year for x in dates_t10]
years_t11=[x.year for x in dates_t11]

years_tc1=[x.year for x in dates_tc]
years_ar=[x.year for x in dates_ar]
years_other=[x.year for x in dates_other]

In [ ]:
print(dates_ar)

In [ ]:
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(20,12))
ax=plt.subplot(1,1,1,projection=proj_map)

years_zipped=list(zip(years_tc1,dates_tc))
dates_tc_first=[x[1] for x in years_zipped if x[0]<=1999]
dates_tc_second=[x[1] for x in years_zipped if x[0]>=2000]
years_tc_first=[x[0] for x in years_zipped if x[0]<=1999]
years_tc_second=[x[0] for x in years_zipped if x[0]>=2000]

years_zipped1=list(zip(years_ar,dates_ar))
dates_ar_first=[x[1] for x in years_zipped1 if x[0]<=1999]
dates_ar_second=[x[1] for x in years_zipped1 if x[0]>=2000]
years_ar_first=[x[0] for x in years_zipped1 if x[0]<=1999]
years_ar_second=[x[0] for x in years_zipped1 if x[0]>=2000]

years_zipped2=list(zip(years_other,dates_other))
dates_other_first=[x[1] for x in years_zipped2 if x[0]<=1999]
dates_other_second=[x[1] for x in years_zipped2 if x[0]>=2000]
years_other_first=[x[0] for x in years_zipped2 if x[0]<=1999]
years_other_second=[x[0] for x in years_zipped2 if x[0]>=2000]

from matplotlib.collections import LineCollection
import matplotlib.patches as mpatches
from matplotlib import pyplot

index_list_date=[]
dates_choose=dates_other_first
years_choose=years_other_first
title='Other: 1979-1999'
for d in range(len(dates_choose)):
#for d in range(2):
    date=dates_choose[d]#date is at 18Z automatically. 
    print(date)
    date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT selection

    year=date.year
    check_lat_tc=lat_tc.where(years_tc==year,drop=True)
    check_lon_tc=lon_tc.where(years_tc==year,drop=True)
    storm_indices=check_lat_tc.storm.values.tolist()
    
    index_list=[]
    for date in date_range:
        date=pd.to_datetime(date)
        month=date.month
        day=date.day
        hour=date.hour
        
        check_lat_tc1=check_lat_tc.where(months_tc==month,drop=True)
        check_lat_tc1=check_lat_tc1.where(days_tc==day,drop=True)
        check_lat_tc1=check_lat_tc1.where(hours_tc==hour,drop=True)
            
        check_lon_tc1=check_lon_tc.where(months_tc==month,drop=True)
        check_lon_tc1=check_lon_tc1.where(days_tc==day,drop=True)
        check_lon_tc1=check_lon_tc1.where(hours_tc==hour,drop=True) 
        
        check_lat_tc_list=[]
        check_lon_tc_list=[]
        for l in check_lat_tc1.storm.values:
            print(l)
            check_lat_tc_values=check_lat_tc1.sel(storm=l).values.tolist()
            check_lat_tc_values=[x for x in check_lat_tc_values if x>0]

            check_lon_tc_values=check_lon_tc1.sel(storm=l).values.tolist()
            check_lon_tc_values=[x for x in check_lon_tc_values if x>-360]
            
            if len(check_lon_tc_values)>0:
                lon_tc_single=check_lon_tc_values[0]
                lat_tc_single=check_lat_tc_values[0]
                
                if 116<=lon_tc_single<=126 and 18<=lat_tc_single<=29.5:
                    index_list.append(l)
                
                
    index_list=list(set(index_list))
    index_list_date.append(index_list)
        
print(index_list_date)
#sys.exit()
print('A')
for d in range(len(dates_choose)):
#for d in range(len(dates_unique)):
    print(d)
    date=dates_choose[d]#date is at 6Z automatically. 
    date_day_end=date+dt.timedelta(hours=24)
    date_start=date_day_end+dt.timedelta(days=-7)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(28)]
    date_range=pd.DatetimeIndex(date_range).values
    year=years_choose[d]
    #index_list=index_list_date[d]
    
    check_lat_tc=lat_tc.where(years_tc==year,drop=True)
    check_lon_tc=lon_tc.where(years_tc==year,drop=True)
    check_months_tc=months_tc.where(years_tc==year,drop=True)
    check_days_tc=days_tc.where(years_tc==year,drop=True)
    check_hours_tc=hours_tc.where(years_tc==year,drop=True)
    check_nature_tc=nature_tc.where(years_tc==year,drop=True)
    check_wind_tc=wind_tc.where(years_tc==year,drop=True)
    segs=[]
    for i in check_lat_tc.storm.values:
        #if i in index_list:
        if 1==1:
            storm_lat=check_lat_tc.sel(storm=i).values.tolist()
            storm_lon=check_lon_tc.sel(storm=i).values.tolist()
            storm_month=check_months_tc.sel(storm=i).values.tolist()
            storm_day=check_days_tc.sel(storm=i).values.tolist()
            storm_hour=check_hours_tc.sel(storm=i).values.tolist()
            storm_wind=check_wind_tc.sel(storm=i).values.tolist()
            storm_nature=check_nature_tc.sel(storm=i).values.tolist()
            storm_month=[x for x in storm_month if x>=0]
            storm_day=[x for x in storm_day if x>=0]
            storm_hour=[x for x in storm_hour if x>=0]
            storm_date=[pd.to_datetime(dt.datetime(year,int(x),int(y),int(z))) for x,y,z in zip(storm_month,storm_day,storm_hour)]
            storm_wind=[x for x in storm_wind if x>=0]
            zipped_latlon=list(zip(storm_lat,storm_lon,storm_date,storm_wind,storm_nature))
            select_week=[x for x in zipped_latlon if x[2] in date_range]
            select_lat=[x[0] for x in select_week]
            select_lon=[x[1] for x in select_week]
            select_wind=[x[3] for x in select_week]
            select_nature=[x[4] for x in select_week]
            if len(select_week)>0:
                for j in range(len(select_week)-1):
                    lat_1=select_lat[j]
                    lon_1=select_lon[j]
                    lat_2=select_lat[j+1]
                    lon_2=select_lon[j+1]
                    starting_wind=select_wind[j]#in kts
                    starting_nature=select_nature[j]
                    #now choose color based on category
                    if starting_nature==1:
                        if starting_wind<=33:
                            color='b'
                        if 34<=starting_wind<=63:
                            color='g'
                        if 64<=starting_wind<=82:
                            color='yellow'
                        if 83<=starting_wind<=95:
                            color='orange'
                        if 96<=starting_wind<=112:
                            color='r'
                        if 113<=starting_wind<=136:
                            color='darkred'
                        if starting_wind>=137:
                            color='m'
                    else:
                        color='k'

                    seg=[[lon_1,lat_1],[lon_2,lat_2]]
                    ax.plot([lon_1,lon_2],[lat_1,lat_2],transform=ccrs.PlateCarree(),linewidth=5.0,color=color)
                    segs.append(seg)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
ax.set_extent([95,155,5,45],crs=ccrs.PlateCarree())

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [85,90,95,100,105,110,115,120,125,130,135,140,145,150,155,160]
yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
ax.tick_params(labelsize=20)
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,alpha=0)
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.top_labels=False
g1.right_labels=False

props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
#ax.text(0.93, 0.985,'n='+str(len(dates_choose)), transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props,zorder=30)
ax.text(0.912, 0.985,'n='+str(len(dates_choose)), transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props,zorder=30)
ax.set_title(title,fontsize=38,pad=10)#2000 - 2019; 1979-1999

patch1 = mpatches.Patch(color='b', alpha=1,label='TD')
patch2 = mpatches.Patch(color='g', alpha=1,label='TS')
patch3 = mpatches.Patch(color='yellow', alpha=1,label='Cat 1')
patch4 = mpatches.Patch(color='orange', alpha=1,label='Cat 2')
patch5 = mpatches.Patch(color='r', alpha=1,label='Cat 3')
patch6 = mpatches.Patch(color='darkred', alpha=1,label='Cat 4')
patch7 = mpatches.Patch(color='m', alpha=1,label='Cat 5')
patch8 = mpatches.Patch(color='k', alpha=1,label='Non-tropical')

legend1=pyplot.legend(handles=[patch1,patch2,patch3,patch4,patch5,patch6,patch7,patch8],loc='upper left',prop={'size':20},fancybox=True, framealpha=1.0)
pyplot.gca().add_artist(legend1)

plt.show()

In [ ]:
fig.savefig(dir+'taiwan_tc_halves_paper_6.png')

In [ ]:
print(dates_ar_first)